# custom module

In [1]:
import torch
from torch import nn
from torch.utils import data
import numpy as np

## 1.定义模型

我们的目标是使用前n天的(click, pv)来预测新一天的ctr:<br>
$$\tilde{ctr}_{n + 1} = \frac{\sum_{i=1}^{n}click_{i}*param_{i} }{\sum_{i=1}^{n}pv_{i} * param_{i} } $$
所求为$param_{i}$。在ctr很稳定的理想状态，$param_{i}=1$。<br>
模型图示如下：

![jupyter](./custom_module.svg)

In [2]:
class TimeDecayModule(nn.Module):
    def __init__(self, num_input):
        super(TimeDecayModule, self).__init__()
        self.linear = nn.Linear(num_input, 1, bias=False)
        
    def forward(self, x):
        click = x[:, 0, :]
        pv = x[:, 1, :]
        weighted_click = self.linear(click)
        weighted_pv = self.linear(pv)
        return weighted_click / weighted_pv

模型参数初始化为1

In [3]:
model = TimeDecayModule(20)
torch.nn.init.ones_(model.linear.weight)

Parameter containing:
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.]], requires_grad=True)

## 2.准备训练所需四要素

1、模型 model<br>
2、数据 train_iter<br>
3、损失函数 loss<br>
4、优化器 optimizer

In [4]:
x_tensor = torch.tensor(np.load("custom_x.npy"), dtype=torch.float32)
y_tensor = torch.tensor(np.load("custom_y.npy").reshape(-1, 1), dtype=torch.float32)
data_set = data.TensorDataset(x_tensor, y_tensor)
train_iter = data.DataLoader(data_set, batch_size=10, shuffle=True)
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.3)

## 3.训练

In [5]:
epoch_num = 200
for epoch in range(epoch_num):
    total_loss = 0.0
    total_num = 0.0
    for x, y in train_iter:
        y_hat = model(x)
        l = loss(y_hat, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        total_loss += l
        total_num += y.shape[0]
    if (epoch + 1) % 10 == 0:
        print(f"epoch {epoch + 1}: {total_loss / total_num}")

epoch 10: 0.0005044274730607867
epoch 20: 0.0005039040697738528
epoch 30: 0.0005026823491789401
epoch 40: 0.000502226350363344
epoch 50: 0.0005011449684388936
epoch 60: 0.0004997661453671753
epoch 70: 0.0004992382600903511
epoch 80: 0.0004982589744031429
epoch 90: 0.0004970498266629875
epoch 100: 0.0004964246181771159
epoch 110: 0.0004960703663527966
epoch 120: 0.0004945984692312777
epoch 130: 0.0004938296624459326
epoch 140: 0.0004925548564642668
epoch 150: 0.0004922162042930722
epoch 160: 0.0004907636321149766
epoch 170: 0.0004895849851891398
epoch 180: 0.0004888690309599042
epoch 190: 0.0004876459133811295
epoch 200: 0.00048713872092776


## 4.提取所需参数

In [6]:
for p in model.parameters():
    print(p)

Parameter containing:
tensor([[1.2367, 1.3100, 0.9641, 1.3109, 1.3259, 1.3914, 1.4736, 1.1585, 1.1422,
         1.0964, 0.9547, 0.7478, 0.6083, 0.5517, 0.4028, 0.7264, 0.3162, 0.5645,
         0.4895, 0.9492]], requires_grad=True)
